# Proyecto 3
#### Integrantes
* Dayana Valentina Gonzalez Vargas 
* Maria Fernanda Rodriguez Conde

### Objetivo del proyecto
Pronosticar los rendimientos a corto plazo (rendimientos residuales de 15 minutos) en 14 criptomonedas populares.

## Concepto importantes 
### 1. Mercado de las criptomonedas
Las criptomonedas se han convertido en mercado extremadamente popular y volátil, que ofrece enormes rendimientos( así como perdidas) a los inversionistas. Tenemos como criptodivisas el Bitcoin (BTC), Ether (ETH) ó Dogecoin (DOGE).
### 2. Prevision de la rentabilidad
Con el fin de predecir como se comportaran los precios en el futuro, es decir, predecir si los precios subirán ó bajarán y en que medida los rendimientos de los activos.Se deben tener en cuenta conceptos como la extrema volatilidad de los mercados, la naturaleza no estacionaria, la manipulación del mercado y la correlación entre activos con las condiciones de mercado que cambian rapidamente.


## Librerias

In [ ]:
# Librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pandas import Grouper
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.graphics.gofplots import qqplot
from statsmodels.tsa.ar_model import AutoReg as AR
from statsmodels.tsa.arima.model import ARIMA
from datetime import datetime, timedelta
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import grid_search_forecaster
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.statespace.mlemodel import PredictionResultsWrapper
from fbprophet import Prophet
#from prophet import Prophet
from sklearn.linear_model import LinearRegression
import xgboost as xgb
import time

import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plt.rc('figure', figsize=(18, 3))
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.display.max_rows = 20

ImportError: ignored

In [ ]:
pip install statsmodels --upgrade

## Importación Dataset

In [ ]:
data_folder = ""

In [ ]:
crypto_df = pd.read_csv(data_folder + 'train.csv')

In [ ]:
crypto_df.head(10)

In [ ]:
crypto_df.isna().sum()

In [ ]:
crypto_df.shape

# Análisis Exploratorio

#### Funciones necesarias

In [ ]:
# auxiliary function, from datetime to timestamp
totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, "%d/%m/%Y").timetuple()))

In [ ]:
# definimos la funcion del rendimiento logaritmico
def log_return(series, periods=1):
    return np.log(series).diff(periods=periods)

## Reducción de la base de datos
Tomando la base de datos vamos a disminuir la dimensión tomando solamente una criptomoneda y un limite de tiempo determinado.Para tomar una sola criptodivisa miraremos el archivo `asset_details.csv`para ver el indice de la criptomoneda.

In [ ]:
asset_details = pd.read_csv(data_folder + 'asset_details.csv')

In [ ]:
asset_details.head(15)

## Correlación entre todos los activos

In [ ]:
# create dataframe with returns for all assets
all_assets_2021 = pd.DataFrame([])
for asset_id, asset_name in zip(asset_details.Asset_ID, asset_details.Asset_Name):
  asset = crypto_df[crypto_df["Asset_ID"]==asset_id].set_index("timestamp")
  asset = asset.loc[totimestamp('01/06/2021'):totimestamp('20/09/2021')]
  asset = asset.reindex(range(asset.index[0],asset.index[-1]+60,60),method='pad')
  lret = log_return(asset.Close.fillna(0))[1:]
  all_assets_2021 = all_assets_2021.join(lret, rsuffix=asset_name, how="outer")

In [ ]:
plt.imshow(all_assets_2021.corr());
plt.yticks(asset_details.Asset_ID.values, asset_details.Asset_Name.values);
plt.xticks(asset_details.Asset_ID.values, asset_details.Asset_Name.values, rotation='vertical');
plt.colorbar();

Tomaremos IOTA ya que es el de menor peso en el mercado, según estos datos se identifica con el `Asset_ID`  = 8 y su peso es de 1.10.

In [ ]:
Iota = crypto_df[crypto_df['Asset_ID']== 8].set_index('timestamp')


In [ ]:
Iota.head(10)

Tomaremos el Maker ya que es el de menor peso en el mercado, se identifica con el ``Asset_ID`` = 10 y su peso es de 1.10.

In [ ]:
Mak = crypto_df[crypto_df['Asset_ID']== 10].set_index('timestamp')

In [ ]:
Mak.tail(10)

### Ajustando el Dataset

In [ ]:
# auxiliary function, from datetime to timestamp
totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, "%d/%m/%Y").timetuple()))
# Periodo de 2021 junio a septiembre
Iota_mini_2021 = Iota.loc[totimestamp('01/06/2021'):totimestamp('20/09/2021')] 
# Periodo de 2021 septiembre días
Iota_mini_2021_reducido = Iota.loc[totimestamp('18/09/2021'):totimestamp('19/09/2021')]

In [ ]:
# create intervals
# Periodo de 2021 junio a septiembre
Mak_mini_2021 = Mak.loc[totimestamp('01/06/2021'):totimestamp('20/09/2021')] 
# Periodo de 2021 septiembre
Mak_mini_2021_reducido = Mak.loc[totimestamp('18/09/2021'):totimestamp('19/09/2021')]

## Gráfica de vela
Mediante una grafica de vela podremos visualizar los cambios del precio para las criptomodas contra el tiempo, es decir, realizar un analisis de los valores intradía, los cuales son los valores con los que se opera antes del cierre, entonces incluira La apertura, el maximo y minimo precio y el cierre de ese día. El color de las diferentes barras como el rojo, que determina cuando el cierre es inferior a la apertura, en caso contrareo encontraremos el color verde. 

Para nuestro grafico tomaremos las filas más recientes, para ver los ultimos comportamientos del precio contra el tiempo.

**Grafica de vela - Iota**

*Periodo del 2021 (Junio - septiembre)*

In [ ]:
Iota_times = []
for i in range(Iota_mini_2021.shape[0]):
    a=Iota_mini_2021.index[i].astype('datetime64[s]')
    Iota_times.append(a)

In [ ]:
#fig = go.Figure(data=[go.Candlestick(x=Bit_times, open=Bit_mini_2021['Open'], high=Bit_mini_2021['High'], low=Bit_mini_2021['Low'], close=Bit_mini_2021['Close'])])
#fig.show()

**Grafica de vela - Maker**

*Periodo del 2021 (Junio - septiembre)*

In [ ]:
Mak_times = []
for i in range(Mak_mini_2021.shape[0]):
    a=Mak_mini_2021.index[i].astype('datetime64[s]')
    Mak_times.append(a)

In [ ]:
#fig = go.Figure(data=[go.Candlestick(x=Mak_times, open=Mak_mini_2021['Open'], high=Mak_mini_2021['High'], low=Mak_mini_2021['Low'], close=Mak_mini_2021['Close'])])
#fig.show()

Podemos notar que en los ultimos meses se nota una gran diferencia entre el valor del Bitcoin y del Maker, siendo el primero hasta 10 veces mayor que el segundo. Tambien podemos observar que a pesar de la gran diferencia los dos tuvieron cambios muy similares con el paso del tiempo.

## Datos perdidos en el tiempo

In [ ]:
# Original Iota
Iota = Iota.reindex(range(Iota.index[0],Iota.index[-1]+60,60),method='pad')
#Original Maker
Mak = Mak.reindex(range(Mak.index[0],Mak.index[-1]+60,60),method='pad')
# Periodo del 2021 (Junio - septiembre) Iota
Iota_mini_2021 = Iota_mini_2021.reindex(range(Iota_mini_2021.index[0],Iota_mini_2021.index[-1]+60,60),method='pad')
# Periodo del 2021 (Junio - septiembre) Maker
Mak_mini_2021 = Mak_mini_2021.reindex(range(Mak_mini_2021.index[0],Mak_mini_2021.index[-1]+60,60),method='pad')
# Periodo del 2021 (Agosto a septiembre) Iota
Iota_mini_2021_reducido = Iota_mini_2021_reducido.reindex(range(Iota_mini_2021_reducido.index[0],Iota_mini_2021_reducido.index[-1]+60,60),method='pad')
# Periodo del 2021 (Agosto a septiembre) Maker
Mak_mini_2021_reducido = Mak_mini_2021_reducido.reindex(range(Mak_mini_2021_reducido.index[0],Mak_mini_2021_reducido.index[-1]+60,60),method='pad')


In [ ]:
Iota_times = []
for i in range(Iota_mini_2021.shape[0]):
    b = np.int64(Iota_mini_2021.index[i])
    a= b.astype('datetime64[s]')
    Iota_times.append(a)

In [ ]:
Iota_times2 = []
for i in range(Iota_mini_2021_reducido.shape[0]):
    b = np.int64(Iota_mini_2021_reducido.index[i])
    a= b.astype('datetime64[s]')
    Iota_times2.append(a)

In [ ]:
Mak_times = []
for i in range(Mak_mini_2021.shape[0]):
    b = np.int64(Mak_mini_2021.index[i])
    a= b.astype('datetime64[s]')
    Mak_times.append(a)

In [ ]:
Mak_times2 = []
for i in range(Mak_mini_2021_reducido.shape[0]):
    b = np.int64(Mak_mini_2021_reducido.index[i])
    a= b.astype('datetime64[s]')
    Mak_times2.append(a)

## Visualizacion de los datos 

**Close prices**

Comenzamos visualizando el precio cerrado realizando una comparación entre ambas criptomonedas. 

In [ ]:
f = plt.figure(figsize=(15,4))

ax = f.add_subplot(121)
plt.plot(Iota['Close'], label='close')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Price close')
plt.title("IOTA")

ax2 = f.add_subplot(122)
ax2.plot(Mak['Close'], color='red', label='close')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Price close')
plt.title("Maker")

plt.tight_layout()
plt.show()

Podemos observar que en el precio cerrado ambas presentan Un comportamiento similiar ente el periodo de tiempo \[1.54 - 1.61\], Notamos tambien que Maker tiene una escala de precio cerra mayor a Iota, A pesar de que ambos tienen el mismo peso en el mercado.

### Grafica de cierre en el 2021
Realizaremos la misma graficas anteriores pero ahora en los periodos de tiempo desde junio hasta septiembre del año 2021.

In [ ]:
f = plt.figure(figsize=(15,4))

ax = f.add_subplot(121)
plt.plot(Iota_times, Iota_mini_2021['Close'], label='close')
plt.legend()
plt.xlabel('Time')
plt.ylabel('price Close')
plt.title("IOTA")

ax2 = f.add_subplot(122)
ax2.plot(Mak_times, Mak_mini_2021['Close'], color='red', label='close')
plt.legend()
plt.xlabel('Time')
plt.ylabel('price Close')
plt.title("Maker")

plt.tight_layout()
plt.show()

**Open prices**

Visualizamos los precios abiertos de ambas criptomonedas con respecto al tiempo.

In [ ]:
f = plt.figure(figsize=(15,4))

ax = f.add_subplot(121)
plt.plot(Iota['Open'], label='Open')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Price open')
plt.title("IOTA")

ax2 = f.add_subplot(122)
ax2.plot(Mak['Open'], color='red', label='Open')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Price open')
plt.title("Maker")

plt.tight_layout()
plt.show()

Notemos que las graficas de apertura de las diferentes criptomonedas tienen el mismo comportamiento que las graficas de cierre.

### Grafica de apertura en el 2021
Realizaremos la misma graficas anteriores pero ahora en los periodos de tiempo desde junio hasta septiembre del año 2021.

In [ ]:
f = plt.figure(figsize=(15,4))

ax = f.add_subplot(121)
plt.plot(Iota_times, Iota_mini_2021['Open'], label='Open')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Price open ')
plt.title("IOTA")

ax2 = f.add_subplot(122)
ax2.plot(Mak_times, Mak_mini_2021['Open'], color='red', label='Open')
plt.legend()
plt.xlabel('Time')
plt.ylabel('price open')
plt.title("Maker")

plt.tight_layout()
plt.show()

**High Prices**

Ahora visualizaremos los precios mas altos que han tenido ambas criptomonedas con respecto al tiempo.

In [ ]:
f = plt.figure(figsize=(15,4))

ax = f.add_subplot(121)
plt.plot(Iota['High'], label='High')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Price High')
plt.title("IOTA")

ax2 = f.add_subplot(122)
ax2.plot(Mak['High'], color='red', label='High')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Price High')
plt.title("Maker")

plt.tight_layout()
plt.show()

Observamos un crecimiento en el precio de ambas criptomonedas dentro del ultimo periodo del tiempo.Además, los valores más altos se centran en los precios 3.0 para la criptomoneda IOTA y 10000 para la criptomoneda Maker. 

### Grafica de precios altos en el 2021
Realizaremos la misma graficas anteriores pero ahora en los periodos de tiempo desde junio hasta septiembre del año 2021.

In [ ]:
f = plt.figure(figsize=(15,4))

ax = f.add_subplot(121)
plt.plot(Iota_times, Iota_mini_2021['High'], label='High')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Price High')
plt.title("IOTA")

ax2 = f.add_subplot(122)
ax2.plot(Mak_times, Mak_mini_2021['High'], color='red', label='High')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Price High')
plt.title("Maker")

plt.tight_layout()
plt.show()

Observamos que en el rango de junio a agosto hubo una variación de precios entre ambas monedas pero sus valores más altos son reguistrados en mes de septiembre del año 2021. 

## Grafica Precios medios ponderados
Realizaremos la grafica de ambas criptomonedas con respecto a sus precios medios ponderados entre los meses de junio a septiembre del 2021

In [ ]:
f = plt.figure(figsize=(15,4))

ax = f.add_subplot(121)
plt.plot(Iota_times, Iota_mini_2021['VWAP'], label='VWAP')
plt.legend()
plt.xlabel('Time')
plt.ylabel('VWAP')
plt.title("IOTA")

ax2 = f.add_subplot(122)
ax2.plot(Mak_times, Mak_mini_2021['VWAP'], color='red', label='VWAP')
plt.legend()
plt.xlabel('Time')
plt.ylabel('VWAP')
plt.title("Maker")

plt.tight_layout()
plt.show()

Notemos que todas las graficas anteriores tienen un comportamiento estacionario  con respecto al tiempo, debido a que no se genera como tal una tendecia se crecimiento o decrecimiento. Como tambien podemos darnos cuenta estas graficas son muy similares.

## ¿ Estacionaria ó no?
A partir de las graficas anteriores notamos que tienen un comportamiento similar, ahora visualizaremos si son estacionarias o no, en caso de que no sean estacionaria recortaremos un poco el tiempo para lograr una señar estacionaria. Visualizaremos las variables `Open` y `VWAP` para el periodo de junio a septiembre del 2021, para nuestras criptomonedas trabajades anteriormente.

In [ ]:
# Prueba Augmented Dickey-Fuller
def Check_Estacionaria (Serie):
    result = adfuller(Serie)
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
            print('\t%s: %.3f' % (key, value))

    if (result[1] <= 0.05) & (result[4]['5%'] > result[0]):
        print("\u001b[32mStationary\u001b[0m")
    else:
        print("\x1b[31mNon-stationary\x1b[0m")

###  IOTA

In [ ]:
Open_Iota1 = pd.Series(Iota_mini_2021['Open'], index=Iota_mini_2021.index)
# Prueba Augmented Dickey-Fuller
Check_Estacionaria(Open_Iota1)

Notemos que la serie conformada con los precios de apertura de la criptomoneda IOTA, no tiene un comportamiento estacionario como se habia pensado incialmente, debido que al realizar esta prueba de hipotesis contamos con un valor p demasiado grande para ese valor estadistico. 

In [ ]:
Vwap_Iota1 = pd.Series(Iota_mini_2021['VWAP'], index=Iota_mini_2021.index)
# Prueba Augmented Dickey-Fuller
Check_Estacionaria(Vwap_Iota1)

Igual que el anterior tenemos que la serie conformada con la variable `VWAP` tiene un comportamiento no estacionario, debido a que tiene un valor p demasiado grande y un valor estadistico más grande que los valores criticos. Como ambos test nos demuestran que aún tenemos una serie no estacionaria, tomaremos una nueva con los tiempos más reducidos.

In [ ]:
Open_Iota2 = pd.Series(Iota_mini_2021_reducido['Open'], index=Iota_mini_2021_reducido.index)
# Prueba Augmented Dickey-Fuller
Check_Estacionaria(Open_Iota2)

In [ ]:
plt.plot(Iota_times2, Iota_mini_2021_reducido['Open'], label='Open')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Precio de apertura')
plt.title("2021(septiembre)")

Notemos que al reducir mucho más la base de datos podemos obtener una seria que es estacionaria con un valor p de 0.009 y un valor estadistico de -3.4395

In [ ]:
Vwap_Iota2 = pd.Series(Iota_mini_2021_reducido['VWAP'], index=Iota_mini_2021_reducido.index)
# Prueba Augmented Dickey-Fuller
Check_Estacionaria(Vwap_Iota2)

In [ ]:
plt.plot(Iota_times2, Iota_mini_2021_reducido['VWAP'], label='VWAP')
plt.legend()
plt.xlabel('Time')
plt.ylabel('VWAP')
plt.title("2021(septiembre)")

Notemos que como ambas graficas tienen un comportamiento similar, se puede afirmar que las series conformadas por `VWAP`,`Open` y `Close` serán series estacionarias para ese periodo de tiempo.

### Maker

In [ ]:
Open_Mak1 = pd.Series(Mak_mini_2021['Open'], index=Mak_mini_2021.index)
# Prueba Augmented Dickey-Fuller
Check_Estacionaria(Open_Mak1)

In [ ]:
Vwap_Mak1 = pd.Series(Mak_mini_2021['VWAP'], index=Mak_mini_2021.index)
# Prueba Augmented Dickey-Fuller
Check_Estacionaria(Vwap_Mak1)

In [ ]:
Open_Mak2 = pd.Series(Mak_mini_2021_reducido['Open'], index=Mak_mini_2021_reducido.index)
# Prueba Augmented Dickey-Fuller
Check_Estacionaria(Open_Mak2)

In [ ]:
plt.plot(Mak_times2, Mak_mini_2021_reducido['Open'], label='Open')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Open')
plt.title("2021(septiembre)")

A pesar de que se recorto el tiempo, la serie sigue teniendo un comportamiento no estacionario, lo que nos indica que no importa cuanto se manipule el tiempo puede llegar a seguir siendo no estacionaria.

## Rendimiento logaritmico de cada activo

Para analizar la evolucion de los precios de un activo podemos tratar la diferencia de precios. Tenemos como consecuencia que como el precio de los activos es tan volatil y tiene escalas diferentes no es muy facil hacer una comparacion, para esto podemos utilizar rendimientos logaritmicos ya que son aditivos a lo largo del tiempo, por lo cual mientras los rendimientos regulares no pueden ser inferiores a -100%, los rendimientos logaritmicos no estan acotados.  
Para entender el procedimiento, este toma el logaritmo de la relacion entre dos precios consecutivos.

*Ahora, mostraremos el ruido del precio de dos criptomonedas, con respecto al periodo de junio hasta septiembre.*

### Log_Return - Close

In [ ]:
lret_Iota = log_return(Iota_mini_2021_reducido.Close)[1:]
lret_Mak = log_return(Mak_mini_2021_reducido.Close)[1:]
lret_Iota.rename('lret_Iota', inplace=True)
lret_Mak.rename('lret_Mak', inplace=True)

plt.figure(figsize=(8,4))
plt.plot( lret_Iota, label="lret_Iota");
plt.plot( lret_Mak,label='lret_Mak');
plt.xlabel('Time')
plt.ylabel('Log return')
plt.legend()
plt.show()

### Log_Return- Open

In [ ]:
lret_Iota = log_return(Iota_mini_2021_reducido.Open)[1:]
lret_Mak = log_return(Mak_mini_2021_reducido.Open)[1:]
lret_Iota.rename('lret_Iota', inplace=True)
lret_Mak.rename('lret_Mak', inplace=True)

plt.figure(figsize=(8,4))
plt.plot( lret_Iota, label="lret_Iota");
plt.plot( lret_Mak,label='lret_Mak');
plt.xlabel('Time')
plt.ylabel('Log return')
plt.legend()
plt.show()

### Log_Return- VWAP

In [ ]:
lret_Iota = log_return(Iota_mini_2021_reducido.VWAP)[1:]
lret_Mak = log_return(Mak_mini_2021_reducido.VWAP)[1:]
lret_Iota.rename('lret_Iota', inplace=True)
lret_Mak.rename('lret_Mak', inplace=True)

plt.figure(figsize=(8,4))
plt.plot( lret_Iota, label="lret_Iota");
plt.plot( lret_Mak,label='lret_Mak');
plt.xlabel('Time')
plt.ylabel('Log return')
plt.legend()
plt.show()

## Correlacion entre activos

In [ ]:
lret_Iota_long = log_return(Iota.Close)[1:]
lret_Mak_long = log_return(Mak.Close)[1:]
lret_Iota_long.rename('lret_Iota', inplace=True)
lret_Mak_long.rename('lret_Mak', inplace=True)
two_assets = pd.concat([lret_Iota_long, lret_Mak_long], axis=1)


corr_time = two_assets.groupby(two_assets.index//(10000*60)).corr().loc[:,"lret_Iota"].loc[:,"lret_Mak"]

corr_time.plot();
plt.xticks([])
plt.ylabel("Correlation")
plt.title("Correlation between Iota and Maker over time");

La correlación entre ambas criptomonedas surge en los ultimos periodos de tiempo, ya que inicialmente su correlación era cercana a cero.

## Generación de los modelos y las predicciones.

Se realizara los modelos para las criptomonedas IOTA y Maker para obtener predicciones futuras, teniendo en cuanta solamente los meses de junio a septiembre del 2021.

## Generar Feactures

In [ ]:
def upper_shadow (data):
    upp = data['High'] - np.maximum(data['Close'],data['Open'])
    return upp
    
def lower_shadow (data): 
    low = np.minimum(data['Close'],data['Open'])- data['Low']
    return low



In [ ]:
# Utilizamos la función fillna(0) Con log_return
X_Iota = pd.concat([log_return(Iota_mini_2021_reducido.VWAP,periods=5), log_return(Iota_mini_2021_reducido.VWAP,periods=1).abs(), 
               upper_shadow(Iota_mini_2021_reducido), lower_shadow(Iota_mini_2021_reducido),log_return(Iota_mini_2021_reducido.Open,periods=1),log_return(Iota_mini_2021_reducido.Close,periods=1)], axis=1).fillna(0)

Y_Iota =Iota_mini_2021_reducido[['Target']].fillna(0)

X_Mak = pd.concat([log_return(Mak_mini_2021_reducido.VWAP,periods=5), log_return(Mak_mini_2021_reducido.VWAP,periods=1).abs(), 
               upper_shadow(Mak_mini_2021_reducido), lower_shadow(Mak_mini_2021_reducido),log_return(Mak_mini_2021_reducido.Open,periods=1),log_return(Mak_mini_2021_reducido.Close,periods=1)], axis=1).fillna(0)

Y_Mak =Mak_mini_2021_reducido[['Target']].fillna(0)


### Generador Shift

In [ ]:
# Shift Iota
y_shift_per5 = Y_Iota.shift(periods = 5)
y_shift_per1 = Y_Iota.shift()
#Shift Maker
y_shift_M_per5 = Y_Mak.shift(periods = 5)
y_shift_M_per1 = Y_Mak.shift()

In [ ]:
X_Iota = X_Iota.assign(Date=Iota_times2)
X_Mak = X_Mak.assign(Date = Mak_times2)
Y_Iota = Y_Iota.assign(Date=Iota_times2).to_numpy()
Y_Mak = Y_Mak.assign(Date = Mak_times2).to_numpy()

In [ ]:
X_Iota = X_Iota.assign(y_shift_per5=y_shift_per5)
X_Mak = X_Mak.assign(y_shift_per5=y_shift_M_per5)

In [ ]:
X_Iota = X_Iota.assign(y_shift_per1=y_shift_per1).fillna(0).to_numpy()
X_Mak = X_Mak.assign(y_shift_per1=y_shift_M_per1).fillna(0).to_numpy()

In [ ]:
X_Iota = pd.DataFrame(X_Iota,columns = [ 'VWAP', '|VWAP|', 'upper_shadow', 'lower_shadow', 'Open', 'Close','Date','y_shift_per5','y_shift_per1'] )
Y_Iota = pd.DataFrame(Y_Iota,columns = ['Target','Date'])

X_Mak = pd.DataFrame(X_Mak,columns = [ 'VWAP', '|VWAP|', 'upper_shadow', 'lower_shadow', 'Open', 'Close','Date','y_shift_per5','y_shift_per1'] )
Y_Mak = pd.DataFrame(Y_Mak,columns = ['Target','Date'])


### Partición en conjunto entrenamiento y prueba

In [ ]:
# Partición en conjunto de entrenamiento y de pueba para la criptomoneda IOTA
tscv = TimeSeriesSplit()

for train_index, test_index in tscv.split(X_Iota):
    X_train_Iota, X_test_Iota = X_Iota.loc[train_index], X_Iota.loc[test_index]
    y_train_Iota, y_test_Iota = Y_Iota.loc[train_index], Y_Iota.loc[test_index]    
    

In [ ]:
# Partición en conjunto de entrenamiento y de pueba para la criptomoneda Maker
tscv2 = TimeSeriesSplit()

for train_index, test_index in tscv2.split(X_Mak):
    X_train_Mak, X_test_Mak = X_Mak.loc[train_index], X_Mak.loc[test_index]
    y_train_Mak, y_test_Mak = Y_Mak.loc[train_index], Y_Mak.loc[test_index] 

In [ ]:
X_train_Iota = X_train_Iota.set_index('Date')
X_test_Iota = X_test_Iota.set_index('Date')
X_train_Mak = X_train_Mak.set_index('Date')
X_test_Mak = X_test_Mak.set_index('Date')
y_train_Iota = y_train_Iota.set_index('Date')
y_test_Iota = y_test_Iota.set_index('Date')
y_train_Mak = y_train_Mak.set_index('Date')
y_test_Mak = y_test_Mak.set_index('Date')

## Estandarización 

In [ ]:
scaler = StandardScaler()

X_Iota_train_scaled = scaler.fit_transform(X_train_Iota)
X_Iota_test_scaled = scaler.transform(X_test_Iota)

X_Mak_train_scaled = scaler.fit_transform(X_train_Mak)
X_Mak_test_scaled = scaler.transform(X_test_Mak)

y_train_Iota_scaled = scaler.fit_transform(y_train_Iota)
y_test_Iota_scaled = scaler.fit_transform(y_test_Iota)

y_train_Mak_scaled = scaler.fit_transform(y_train_Mak)
y_test_Mak_scaled = scaler.fit_transform(y_test_Mak)

In [ ]:
X_Iota_train_scaled = pd.DataFrame(X_Iota_train_scaled,columns = [ 'VWAP', '|VWAP|', 'upper_shadow', 'lower_shadow', 'Open', 'Close','y_shift_per5','y_shift_per1'], index=X_train_Iota.index)
X_Iota_test_scaled = pd.DataFrame(X_Iota_test_scaled,columns = [ 'VWAP', '|VWAP|', 'upper_shadow', 'lower_shadow', 'Open', 'Close','y_shift_per5','y_shift_per1'], index=X_test_Iota.index)
X_Mak_train_scaled = pd.DataFrame(X_Mak_train_scaled,columns = [ 'VWAP', '|VWAP|', 'upper_shadow', 'lower_shadow', 'Open', 'Close','y_shift_per5','y_shift_per1'], index=X_train_Mak.index)
X_Mak_test_scaled = pd.DataFrame(X_Mak_test_scaled,columns = [ 'VWAP', '|VWAP|', 'upper_shadow', 'lower_shadow', 'Open', 'Close','y_shift_per5','y_shift_per1'], index=X_test_Mak.index) 
y_train_Iota_scaled = scaler.fit_transform(y_train_Iota)
y_test_Iota_scaled = scaler.fit_transform(y_test_Iota)
y_train_Iota_scaled = pd.DataFrame(y_train_Iota_scaled,columns = ['target'], index=y_train_Iota.index)
y_test_Iota_scaled = pd.DataFrame(y_test_Iota_scaled,columns = ['target'], index=y_test_Iota.index)
y_train_Mak_scaled = pd.DataFrame(y_train_Mak_scaled,columns = ['target'], index=y_train_Mak.index)
y_test_Mak_scaled = pd.DataFrame(y_test_Mak_scaled,columns = ['target'], index=y_test_Mak.index)

## Modelos
Iniciamos tranformando la variable respuesta en una serie.

In [ ]:
Y_Train_Iota = pd.Series(y_train_Iota_scaled.target, index=y_train_Iota_scaled.index)
Y_Test_Iota = pd.Series(y_test_Iota_scaled.target, index=y_test_Iota_scaled.index)

In [ ]:
Y_Train_Mak = pd.Series(y_train_Mak_scaled.target, index=y_train_Mak_scaled.index)
Y_Test_Mak = pd.Series(y_test_Mak_scaled.target, index=y_test_Mak_scaled.index)

## Forescast Autoregressor 
### Criptomoneda - Iota

In [ ]:
f, ax = plt.subplots(nrows=4, ncols=2, figsize=(30,10))


plot_acf(X_Iota_train_scaled['VWAP'], lags=90, ax=ax[0][0],title = "Autocorrelación - Iota(VWAP)")
plot_acf(X_Iota_train_scaled['|VWAP|'], lags=90, ax=ax[0][1],title = "Autocorrelación - Iota(|VWAP|)")
plot_acf(X_Iota_train_scaled['upper_shadow'], lags=90, ax=ax[1][0],title = "Autocorrelación - Iota(upper_shadow)")
plot_acf(X_Iota_train_scaled['lower_shadow'], lags=90, ax=ax[1][1],title = "Autocorrelación - Iota(lower_shadow)")
plot_acf(X_Iota_train_scaled['Close'], lags=90, ax=ax[2][0],title = "Autocorrelación - Iota(Close)")
plot_acf(X_Iota_train_scaled['Open'], lags=90, ax=ax[2][1],title = "Autocorrelación - Iota(Open)")
plot_acf(X_Iota_train_scaled['y_shift_per5'], lags=90, ax=ax[3][0],title = "Autocorrelación - Iota(y_shift(periodo = 5))")
plot_acf(X_Iota_train_scaled['y_shift_per1'], lags=90, ax=ax[3][1],title = "Autocorrelación - Iota(y_shift(periodo = 1)")

plt.tight_layout()
plt.show()

In [ ]:
plot_acf(Y_Train_Iota , lags=90,title = "Autocorrelación - IOTA(Target)") 

plt.tight_layout()
plt.show()

In [ ]:
forecaster_rf = ForecasterAutoreg(
                    regressor=RandomForestRegressor( random_state=123, n_estimators = 100),
                    lags=90)

In [ ]:
forecaster_rf.fit(Y_Train_Iota, exog = X_Iota_train_scaled)

In [ ]:

predicciones = forecaster_rf.predict(steps =len(Y_Test_Iota), exog = X_Iota_test_scaled )


# Se añade el índice temporal a las predicciones
predicciones = pd.Series(data=predicciones, index=Y_Test_Iota.index)
predicciones.head()

### Metricas

In [ ]:
def pcorrelation (y_pred,y_test,moneda):
    print('Test score:' + moneda, f"{np.corrcoef(y_pred, y_test)[0,1]:.2f}")

In [ ]:
error_mse = mean_squared_error(
                y_true = Y_Test_Iota,
                y_pred = predicciones
            )

rmse = np.sqrt(error_mse)
pcorrelation(predicciones,Y_Test_Iota,'IOTA')
print('Test RMSE: %.3f' % rmse)
print(f"Error de test (mse): {error_mse}")

### Criptomoneda - Maker

In [ ]:
f, ax = plt.subplots(nrows=4, ncols=2, figsize=(30,10))


plot_acf(X_Mak_train_scaled['VWAP'], lags=90, ax=ax[0][0],title = "Autocorrelación - Iota(VWAP)")
plot_acf(X_Mak_train_scaled['|VWAP|'], lags=90, ax=ax[0][1],title = "Autocorrelación - Iota(|VWAP|)")
plot_acf(X_Mak_train_scaled['upper_shadow'], lags=90, ax=ax[1][0],title = "Autocorrelación - Iota(upper_shadow)")
plot_acf(X_Mak_train_scaled['lower_shadow'], lags=90, ax=ax[1][1],title = "Autocorrelación - Iota(lower_shadow)")
plot_acf(X_Mak_train_scaled['Close'], lags=90, ax=ax[2][0],title = "Autocorrelación - Iota(Close)")
plot_acf(X_Mak_train_scaled['Open'], lags=90, ax=ax[2][1],title = "Autocorrelación - Iota(Open)")
plot_acf(X_Mak_train_scaled['y_shift_per5'], lags=90, ax=ax[3][0],title = "Autocorrelación - Mak(y_shift(periodo = 5))")
plot_acf(X_Mak_train_scaled['y_shift_per1'], lags=90, ax=ax[3][1],title = "Autocorrelación - Mak(y_shift(periodo = 1)")


plt.tight_layout()
plt.show()

In [ ]:
plot_acf(Y_Train_Mak , lags=90,title = "Autocorrelación - Mak(Target)") 

plt.tight_layout()
plt.show()

In [ ]:
forecaster_rf = ForecasterAutoreg(
                    regressor=RandomForestRegressor(random_state=123),
                    lags=6)

In [ ]:
forecaster_rf.fit(Y_Train_Mak, exog = X_Mak_train_scaled)

In [ ]:

predicciones = forecaster_rf.predict(steps =len(Y_Test_Mak), exog = X_Mak_test_scaled )


# Se añade el índice temporal a las predicciones
predicciones = pd.Series(data=predicciones, index=Y_Test_Mak.index)
predicciones.head()

### Metricas

In [ ]:
error_mse = mean_squared_error(
                y_true = Y_Test_Mak,
                y_pred = predicciones
            )

rmse = np.sqrt(error_mse)
pcorrelation(predicciones,Y_Test_Mak,'Mak')
print('Test RMSE: %.3f' % rmse)
print(f"Error de test (mse): {error_mse}")

## ARIMA Model
### Evaluación de hiperparametros

In [ ]:
def evaluate_arima_model(y_test,y_train,x_test,x_train, arima_order):
    history = [x for x in y_train]
    index = 0
    # make predictions
    predictions = []
    for t in range(len(y_test)):
        model = ARIMA(y_train, exog = x_train, order=arima_order)
        model_fit = model.fit()
        yhat = model_fit.forecast(exog = x_test.iloc[index])[0]
        index += 1
        predictions.append(yhat)
        history.append(y_test.iloc[t])
    # calculate out of sample error
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    return rmse

In [ ]:
def evaluate_models(y_test,y_train,x_test,x_train, p_values, d_values, q_values):
    y_test = y_test.astype('float32')
    y_train = y_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train = x_train.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    rmse = evaluate_arima_model(y_test,y_train,x_test,x_train, order)
                    print(rmse)
                    if rmse < best_score:
                        best_score, best_cfg = rmse, order
                    print('ARIMA%s RMSE=%.3f' % (order,rmse))
                except:
                    continue
    
    print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))

In [ ]:
p_values = [1, 5, 0]
d_values = [1, 2]
q_values = [1, 0]

evaluate_models(Y_Test_Iota,Y_Train_Iota,X_Iota_test_scaled,X_Iota_train_scaled, p_values, d_values, q_values)

## Iota 

In [ ]:
model_Iota_Arima = ARIMA(endog = Y_Train_Iota , exog = X_Iota_train_scaled, order=(10,0,10))
model_Iota_Arima_fit = model_Iota_Arima.fit()
# summary of fit model
print(model_Iota_Arima_fit.summary())

### Predicciones 

In [ ]:
predic=model_Iota_Arima_fit.predict(start =len(Y_Train_Iota),end=len(Y_Train_Iota)+len(Y_Test_Iota)-1,exog = X_Iota_test_scaled)


### Metricas

In [ ]:
error_mse = mean_squared_error(
                y_true = Y_Test_Iota,
                y_pred = predic
            )

rmse = np.sqrt(error_mse)
pcorrelation(predic,Y_Test_Iota,'IOTA')
print('Test RMSE: %.3f' % rmse)
print(f"Error de test (mse): {error_mse}")

### Residuales 

In [ ]:
# line plot of residuals
residuals = pd.DataFrame(model_Iota_Arima_fit.resid)
residuals.plot()
plt.show()

In [ ]:
# Densidad de las residuales
residuals.plot(kind='kde');

In [ ]:
sns.set(color_codes=True)
sns.kdeplot(np.hstack(residuals.values), shade=True, color="r").set(xlim=(-6, 6))

In [ ]:
residuals.describe()

### Rolling Forecast con Arima 

In [ ]:
history = []
index = 0
# make predictions
predictions = []
for t in range(len(Y_Test_Iota)):
    model = ARIMA(Y_Train_Iota, exog = X_Iota_train_scaled, order=arima_order)
    model_fit = model.fit()
    yhat = model_fit.forecast(exog = X_Iota_test_scaled.iloc[index])[0]
    index += 1
    predictions.append(yhat)
    history.append(Y_Test_Iota[t])

### Metricas

In [ ]:
error_mse = mean_squared_error(
                y_true = Y_Test_Iota,
                y_pred = predictions
            )

rmse = np.sqrt(error_mse)
pcorrelation(predictions,Y_Test_Iota,'IOTA')
print('Test RMSE: %.3f' % rmse)
print(f"Error de test (mse): {error_mse}")

### Criptomoneda-Maker

In [ ]:
p_values = [1, 5, 6]
d_values = [0, 1]
q_values = [0, 1]

evaluate_models(Y_Test_Mak,Y_Train_Mak,X_Mak_test_scaled,X_Mak_train_scaled, p_values, d_values, q_values)

Teniendo en cuenta que con la función de evalución de hiperparametros tuvo un mejor RMSE con (1,0,1), realizaremos el modelo ARIMA con estos valores para ver, las residuales y la metrica de correlación.

### Modelo Arima- Criptomoneda Maker

In [ ]:
model_Mak_Arima = ARIMA(endog = Y_Train_Mak , exog = X_Mak_train_scaled, order=(1,0,1))
model_Mak_Arima_fit = model_Mak_Arima.fit()
# summary of fit model
print(model_Mak_Arima_fit.summary())

### Predicciones

In [ ]:
predic=model_Mak_Arima_fit.predict(start =len(Y_Train_Mak),end=len(Y_Train_Mak)+len(Y_Test_Mak)-1,exog = X_Mak_test_scaled)


### Metrica

In [ ]:
error_mse = mean_squared_error(
                y_true = Y_Test_Mak,
                y_pred = predic
            )

rmse = np.sqrt(error_mse)
pcorrelation(predic,Y_Test_Mak,'Maker')
print('Test RMSE: %.3f' % rmse)
print(f"Error de test (mse): {error_mse}")

### Residuales 

In [ ]:
# line plot of residuals
residuals = pd.DataFrame(model_Mak_Arima_fit.resid)
residuals.plot()
plt.show()

In [ ]:
# Densidad de las residuales
residuals.plot(kind='kde');

In [ ]:
sns.set(color_codes=True)
sns.kdeplot(np.hstack(residuals.values), shade=True, color="r").set(xlim=(-4, 4))

In [ ]:
residuals.describe()

## Modelo SARIMA
Para los siguientes modelos tomaremos el mismo order que nos dio en los hiperparametros de ARIMA.

### Criptomoneda - IOTA

In [ ]:
model_Iota_sarima = SARIMAX(endog = Y_Train_Iota , exog = X_Iota_train_scaled, order=(5,1,0))
res = model_Iota_sarima.fit(disp=False)
print(res.summary())

### Predicciones

In [ ]:
predict = res.get_prediction(start =len(Y_Train_Iota),end=len(Y_Train_Iota)+len(Y_Test_Iota)-1,exog = X_Iota_test_scaled)
predict_ci = predict.conf_int()

In [ ]:
# Graph
fig, ax = plt.subplots(figsize=(9,4))
npre = 4
ax.set(title='Criptomoneda - Iota', xlabel='Date', ylabel='Target')

# Plot data points
Y_Test_Iota.plot(ax=ax, style='o', label='Observed')

# Plot predictions
predict.predicted_mean.plot(ax=ax, style='r--', label='One-step-ahead forecast')
ci = predict_ci
ax.fill_between(ci.index, ci.iloc[:,0], ci.iloc[:,1], color='r', alpha=0.1)

legend = ax.legend()

In [ ]:
predict = res.predict(start =len(Y_Train_Iota),end=len(Y_Train_Iota)+len(Y_Test_Iota)-1,exog = X_Iota_test_scaled)

### Metricas

In [ ]:
error_mse = mean_squared_error(
                y_true = Y_Test_Iota,
                y_pred = predict
            )

rmse = np.sqrt(error_mse)
pcorrelation(predict,Y_Test_Iota,'Iota')
print('Test RMSE: %.3f' % rmse)
print(f"Error de test (mse): {error_mse}")

### Criptomoneda - Maker

In [ ]:
model_Mak_sarima = SARIMAX(endog = Y_Train_Mak , exog = X_Mak_train_scaled, order=(5,1,0))
res = model_Mak_sarima .fit(disp=False)
print(res.summary())

### Predicciones

In [ ]:
predict = res.get_prediction(start =len(Y_Train_Mak),end=len(Y_Train_Mak)+len(Y_Test_Mak)-1,exog = X_Mak_test_scaled)
predict_ci = predict.conf_int()

In [ ]:
# Graph
fig, ax = plt.subplots(figsize=(9,4))
npre = 4
ax.set(title='Criptomoneda - Maker', xlabel='Date', ylabel='Target')

# Plot data points
Y_Test_Mak.plot(ax=ax, style='o', label='Observed')

# Plot predictions
predict.predicted_mean.plot(ax=ax, style='r--', label='One-step-ahead forecast')
ci = predict_ci
ax.fill_between(ci.index, ci.iloc[:,0], ci.iloc[:,1], color='r', alpha=0.1)

legend = ax.legend()

In [ ]:
predict = res.predict(start =len(Y_Train_Mak),end=len(Y_Train_Mak)+len(Y_Test_Mak)-1,exog = X_Mak_test_scaled)

### Metricas

In [ ]:
error_mse = mean_squared_error(
                y_true = Y_Test_Mak,
                y_pred = predict
            )

rmse = np.sqrt(error_mse)
pcorrelation(predict,Y_Test_Mak,'Maker')
print('Test RMSE: %.3f' % rmse)
print(f"Error de test (mse): {error_mse}")

## Modelo Phophet

### Criptomoneda - Iota 

### Criptomoneda - Maker

## Modelos Adicionales

### Regresión lineal
Realizaremos una regresión lineal normal

### Criptomoneda - Iota 

In [ ]:
lr = LinearRegression()
lr.fit(X_Iota_train_scaled,Y_Train_Iota)
### Predicciones
y_pred_lr_Iota = lr.predict(X_Iota_test_scaled)


In [ ]:
error_mse = mean_squared_error(
                y_true = Y_Test_Iota,
                y_pred =y_pred_lr_Iota
            )

rmse = np.sqrt(error_mse)
pcorrelation(y_pred_lr_Iota,Y_Test_Iota,'Iota')
print('Test RMSE: %.3f' % rmse)
print(f"Error de test (mse): {error_mse}")

### Criptomoneda - Mak 

In [ ]:
lr.fit(X_Mak_train_scaled,Y_Train_Mak)
### Prediciones
y_pred_lr_Mak = lr.predict(X_Mak_test_scaled)

In [ ]:
error_mse = mean_squared_error(
                y_true = Y_Test_Mak,
                y_pred =y_pred_lr_Mak
            )

rmse = np.sqrt(error_mse)
pcorrelation(y_pred_lr_Mak,Y_Test_Mak,'Maker')
print('Test RMSE: %.3f' % rmse)
print(f"Error de test (mse): {error_mse}")

## XGBRegresion

### Criptomoneda- Iota

In [ ]:
xgbr = xgb.XGBRegressor(verbosity=0)
Model_Iota_xgbr=xgbr.fit(X_Iota_train_scaled,Y_Train_Iota)
ypred_iota =Model_Iota_xgbr.predict(X_Iota_test_scaled)

In [ ]:
error_mse = mean_squared_error(
                y_true = Y_Test_Iota,
                y_pred =ypred_iota
            )

rmse = np.sqrt(error_mse)
pcorrelation(ypred_iota,Y_Test_Iota,'Iota')
print('Test RMSE: %.3f' % rmse)
print(f"Error de test (mse): {error_mse}")

### Criptomoneda- Maker

In [ ]:
xgbr = xgb.XGBRegressor(verbosity=0)
Model_Mak_xgbr=xgbr.fit(X_Mak_train_scaled,Y_Train_Mak)
ypred_Mak =Model_Iota_xgbr.predict(X_Mak_test_scaled)

In [ ]:
error_mse = mean_squared_error(
                y_true = Y_Test_Mak,
                y_pred =ypred_Mak
            )

rmse = np.sqrt(error_mse)
pcorrelation(ypred_Mak,Y_Test_Mak,'Maker')
print('Test RMSE: %.3f' % rmse)
print(f"Error de test (mse): {error_mse}")

# Bibliografia
* https://www.kaggle.com/c/g-research-crypto-forecasting/overview/evaluation
* https://www.kaggle.com/cstein06/tutorial-to-the-g-research-crypto-competition
* https://www.cienciadedatos.net/documentos/py27-forecasting-series-temporales-python-scikitlearn.html
* https://joaquinamatrodrigo.github.io/skforecast/api/ForecasterAutoreg.html#skforecast.ForecasterAutoreg.ForecasterAutoreg.get_coef
* https://facebook.github.io/prophet/docs/non-daily_data.html
* https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima.model.ARIMA.html